In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'ht

In [24]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in one of the JSON examples provided:"
link_system_prompt += """
Example 1
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}

Example 2 
{
    "links": [
        {"Domain": "About", "url": "https://full.url/goes/here/about"},
        {"Domain": "Specilization": "url": "https://another.full.url/careers"}
    ]
}
"""

In [25]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in one of the JSON examples provided:
Example 1
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}

Example 2 
{
    "links": [
        {"Domain": "About", "url": "https://full.url/goes/here/about"},
        {"Domain": "Specilization": "url": "https://another.full.url/careers"}
    ]
}



In [26]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [27]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2024/12/21/

In [28]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [29]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

# anthropic_page = Website("https://anthropic.com")
# anthropic_page.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/deepseek-ai/DeepSeek-V3-0324',
 '/Qwen/Qwen2.5-Omni-7B',
 '/manycore-research/SpatialLM-Llama-1B',
 '/ds4sd/SmolDocling-256M-preview',
 '/ByteDance/InfiniteYou',
 '/models',
 '/spaces/ByteDance/InfiniteYou-FLUX',
 '/spaces/enzostvs/deepsite',
 '/spaces/3DAIGC/LHM',
 '/spaces/Trudy/gemini-codrawing',
 '/spaces/Qwen/Qwen2.5-Omni-7B-Demo',
 '/spaces',
 '/datasets/nvidia/Llama-Nemotron-Post-Training-Dataset-v1',
 '/datasets/glaiveai/reasoning-v1-20m',
 '/datasets/FreedomIntelligence/medical-o1-reasoning-SFT',
 '/datasets/a-m-team/AM-DeepSeek-R1-Distilled-1.4M',
 '/datasets/PixelAI-Team/TalkBody4D',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '

In [30]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'documentation', 'url': 'https://huggingface.co/docs'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [32]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [33]:
get_all_details("https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'learn page', 'url': 'https://huggingface.co/learn'}, {'type': 'linkedin page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'discuss forum', 'url': 'https://discuss.huggingface.co'}]}


'Landing page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nPosts\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\ndeepseek-ai/DeepSeek-V3-0324\nUpdated\n2 days ago\n•\n60.5k\n•\n1.96k\nQwen/Qwen2.5-Omni-7B\nUpdated\nabout 24 hours ago\n•\n27.9k\n•\n796\nmanycore-research/SpatialLM-Llama-1B\nUpdated\n8 days ago\n•\n11.6k\n•\n781\nds4sd/SmolDocling-256M-preview\nUpdated\n6 days ago\n•\n48.4k\n•\n1.03k\nByteDance/InfiniteYou\nUpdated\n4 days ago\n•\n465\nBrowse 1M+ models\nSpaces\nRunning\non\nZero\n530\n530\nInfiniteYou-FLUX\n📸\nFlexible Photo Recrafting While Preserving Your Identity\nRunning\n255\n255\nDeepSite\n🐳\nImagine and Share in 1-Click\nRunning\non\nZero\n224\n224\nLHM\n⚡\nLarge Animatab

In [39]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Provide the following sections: Company Overview, Key Offerings, and Customers"

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [40]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [41]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nPosts\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\ndeepseek-ai/DeepSeek-V3-0324\nUpdated\n2 days ago\n•\n60.5k\n•\n1.96k\nQwen/Qwen2.5-Omni-7B\nUpdated\nabout 24 hours ago\n•\n27.9k\n•\n796\nmanycore-research/SpatialLM-Llama-1B\nUpdated\n8 days ago\n•\n11.6k\n•\n781\nds4sd/SmolDocling-256M-preview\nUpdated\n6 days ago\n•\n48.4k\n•\n1.03k\nByteDance/InfiniteYou\nUpdated\n4 days ago\n•\n465\nBrowse 1M+ models\nSpaces\nRunning\non\nZero\n530\n530

In [44]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))
    return result

In [43]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}]}


# Hugging Face Brochure

## Company Overview
Hugging Face is a leading AI community dedicated to building the future of machine learning through collaboration. The platform serves as a hub for developers, researchers, and organizations, enabling them to create, share, and collaborate on a vast array of models, datasets, and applications. Hugging Face is designed to accelerate innovation in AI and machine learning, embracing an open-source ethos that empowers users worldwide. 

## Key Offerings
- **Models:** Explore and contribute to over 1M+ state-of-the-art machine learning models across various modalities, including text, image, video, audio, and 3D.
  
- **Datasets:** Access a rich repository of 250k+ datasets, facilitating a wide range of machine learning tasks and research.
  
- **Spaces:** Develop and deploy applications easily within a user-friendly environment, showcasing over 400k ML applications.
  
- **Enterprise Solutions:** Tailored offerings for businesses seeking advanced security, support, and compute resources, starting at $20/user/month.
  
- **Open Source Community:** Hugging Face spearheads numerous open-source libraries like Transformers, Diffusers, and Tokenizers, providing tools and resources for both research and production.

## Customers
Hugging Face serves more than 50,000 organizations across various sectors, including prominent names such as:
- **Meta**
- **Amazon**
- **Google**
- **Microsoft**
- **Intel**
- **Grammarly**

This diverse clientele demonstrates Hugging Face's commitment to empowering industries through innovative machine learning solutions, and building a collaborative ecosystem for AI development.

---

Join Hugging Face today and become part of a community that is at the forefront of the AI revolution! Explore our platform [here](https://huggingface.co).

In [49]:
system_prompt_trs = "You are an assistant who checks company brochures for translation. You will receive a specoific company brochure and translate to the languahe specified by the user. Respond in Markdown."

def get_brochure_user_translation_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Translate the broshure to Arabic.\n"
    user_prompt += f"{create_brochure(company_name, url)}\n"
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

# print(get_brochure_user_translation_prompt("HuggingFace", "https://huggingface.co"))

def translate_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt_trs},
            {"role": "user", "content": get_brochure_user_translation_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [50]:
translate_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}]}


```markdown
# Hugging Face Company Brochure

## Company Overview
Hugging Face is a pioneering community-driven platform at the forefront of artificial intelligence (AI) and machine learning (ML). The company is committed to democratizing access to advanced machine learning tools and fostering collaboration among developers, researchers, and enterprises globally. Hugging Face serves as an open-source hub where users can create, share, and collaborate on models, datasets, and applications, thereby accelerating the development and deployment of AI-driven solutions.

## Key Offerings
### 1. **Models**
- **Access to Over 1 Million Models**: Hugging Face provides a comprehensive library where users can explore and use a vast array of AI models ranging from text generation to image analysis.
- **State-of-the-art ML frameworks**: The platform supports various leading frameworks, including Transformers, Diffusers, and more, suitable for use with PyTorch, TensorFlow, and JAX.

### 2. **Datasets**
- **Extensive Dataset Collection**: Users can browse and access over 250,000 datasets tailored for diverse machine learning tasks, ensuring comprehensive resources for training and research.

### 3. **Spaces**
- **Interactive AI Applications**: Hugging Face hosts numerous applications where users can experiment with ML models directly through a user-friendly interface. These spaces include various tools for image generation, natural language processing, and more.

### 4. **Enterprise Solutions**
- **Tailored for Businesses**: Hugging Face offers enterprise-grade services with cloud solutions, advanced security features, and dedicated support to empower organizations in deploying customized AI solutions efficiently.

### 5. **Open Source Collaboration**
- Hugging Face is committed to transparency and community involvement, providing a robust open-source ecosystem that includes libraries such as `Transformers`, `Tokenizers`, and `Accelerate`.

## Customers
More than **50,000 organizations** trust Hugging Face to power their machine learning capabilities. Among them are industry leaders like:
- **Google**
  - 974 models, 10.7k followers
- **Microsoft**
  - 365 models, 10.7k followers
- **Amazon**
  - 10 models, 2.92k followers
- **Meta, Grammarly**, and numerous others in various sectors, highlighting the platform's versatility and credibility in driving AI innovation.

Whether you're a developer looking to enhance your ML projects, an investor interested in the future of AI technology, or a prospective recruit eager to join a passionate community, Hugging Face is at the heart of the AI revolution. Join us on this journey to build the future!
```


```markdown
# كتيب شركة Hugging Face

## لمحة عامة عن الشركة
تُعتبر Hugging Face منصة رائدة تعتمد على المجتمع في مقدمة الذكاء الاصطناعي (AI) وتعلم الآلة (ML). تلتزم الشركة بديمقراطية الوصول إلى أدوات تعلم الآلة المتقدمة وتعزيز التعاون بين المطورين والباحثين والشركات على مستوى العالم. تعمل Hugging Face كمركز مفتوح المصدر حيث يمكن للمستخدمين إنشاء ومشاركة والتعاون في النماذج ومجموعات البيانات والتطبيقات، مما يساهم في تسريع تطوير ونشر الحلول المدفوعة بالذكاء الاصطناعي.

## العروض الرئيسية
### 1. **النماذج**
- **الوصول إلى أكثر من مليون نموذج**: توفر Hugging Face مكتبة شاملة حيث يمكن للمستخدمين استكشاف واستخدام مجموعة واسعة من نماذج الذكاء الاصطناعي تتراوح من توليد النصوص إلى تحليل الصور.
- **إطارات تعلم الآلة المتطورة**: تدعم المنصة مجموعة من الإطارات الرائدة، بما في ذلك Transformers وDiffusers وغيرها، المناسبة للاستخدام مع PyTorch وTensorFlow وJAX.

### 2. **مجموعات البيانات**
- **مجموعة واسعة من مجموعات البيانات**: يمكن للمستخدمين تصفح والوصول إلى أكثر من 250,000 مجموعة بيانات مخصصة لمهام تعلم الآلة المتنوعة، مما يضمن موارد شاملة للتدريب والبحث.

### 3. **المساحات**
- **تطبيقات الذكاء الاصطناعي التفاعلية**: تستضيف Hugging Face العديد من التطبيقات حيث يمكن للمستخدمين تجربة نماذج تعلم الآلة مباشرة من خلال واجهة مستخدم سهلة الاستخدام. تشمل هذه المساحات أدوات متنوعة لتوليد الصور ومعالجة اللغة الطبيعية والمزيد.

### 4. **حلول الشركات**
- **مصممة للشركات**: تقدم Hugging Face خدمات موجهة للأعمال مع حلول سحابية وميزات أمان متقدمة ودعم مخصص لتمكين المنظمات من نشر حلول ذكاء اصطناعي مخصصة بكفاءة.

### 5. **التعاون مفتوح المصدر**
- تلتزم Hugging Face بالشفافية ومشاركة المجتمع، حيث توفر نظام بيئي قوي مفتوح المصدر يشمل مكتبات مثل `Transformers` و `Tokenizers` و `Accelerate`.

## العملاء
تثق أكثر من **50,000 منظمة** في Hugging Face لتعزيز قدراتها في تعلم الآلة. من بين هؤلاء، هناك قادة في الصناعة مثل:
- **جوجل**
  - 974 نموذج، 10.7 ألف متابع
- **مايكروسوفت**
  - 365 نموذج، 10.7 ألف متابع
- **أمازون**
  - 10 نماذج، 2.92 ألف متابع
- **ميتا، جرامرلي** والعديد من الآخرين في مختلف القطاعات، مما يبرز تنوع ومصداقية المنصة في دفع الابتكار في مجال الذكاء الاصطناعي.

سواء كنت مطورًا يتطلع إلى تعزيز مشاريع تعلم الآلة الخاصة بك، مستثمرًا مهتمًا بمستقبل تكنولوجيا الذكاء الاصطناعي، أو مرشحًا محتملًا متحمسًا للانضمام إلى مجتمع مليء بالشغف، فإن Hugging Face هي في قلب ثورة الذكاء الاصطناعي. انضم إلينا في هذه الرحلة لبناء المستقبل!
```

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [24]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [29]:
# stream_brochure("HuggingFace", "https://huggingface.co")
# stream_brochure("British Council", "https://www.gov.uk/world/organisations/british-embassy-tel-aviv")
# stream_brochure("General Motors", "https://www.gm.com/")
stream_brochure("Nvidia", "https://www.nvidia.com/en-eu/")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.nvidia.com/en-eu/about-nvidia/'}, {'type': 'careers page', 'url': 'https://www.nvidia.com/en-eu/about-nvidia/careers/'}, {'type': 'company page', 'url': 'https://www.nvidia.com/en-eu/about-nvidia/executive-insights/'}, {'type': 'company page', 'url': 'https://www.nvidia.com/en-eu/about-nvidia/partners/'}, {'type': 'company page', 'url': 'https://www.nvidia.com/en-eu/research/'}, {'type': 'company page', 'url': 'https://www.nvidia.com/en-eu/foundation/'}, {'type': 'company page', 'url': 'https://www.nvidia.com/en-eu/csr/'}]}



# NVIDIA Company Brochure

## About NVIDIA

NVIDIA is a global leader in artificial intelligence (AI) computing, revolutionizing the way industries leverage technology to harness the power of data. Our expertise spans various domains including gaming, automotive, healthcare, and cloud computing, establishing NVIDIA at the forefront of accelerated computing for modern applications.

### Vision and Mission

NVIDIA's mission is to enhance human productivity through advanced technology. We aim to drive innovation and transform industries by delivering powerful graphics and computing solutions that empower creativity and enable smarter decisions.

---

## Products and Services

### Innovative Offerings

- **Artificial Intelligence Solutions:** From generative AI to intelligent video analytics, NVIDIA is pioneering the future with cutting-edge AI tools and frameworks.
- **Graphics Processing Units (GPUs):** The renowned GeForce and RTX product lines deliver unparalleled graphics performance for gaming and creative applications.
- **Data Center Solutions:** Our advanced networking and cloud solutions power mission-critical applications across industries, ensuring robust and scalable computing environments.

### Key Industries Served

- **Gaming:** Providing gamers with the best graphical experiences through high-performance graphics cards and technologies.
- **Automotive:** Leading innovations in autonomous vehicles and smart transportation systems.
- **Healthcare:** Transforming clinical applications with AI-powered solutions, data analysis, and simulation tools.

---

## Company Culture

At NVIDIA, our culture is founded on collaboration and innovation. We encourage a growth mindset while fostering an inclusive environment where every team member is empowered to contribute their ideas. Our commitment to diversity and sustainability sets the groundwork for a dynamic workplace where creativity flourishes.

### Employee Engagement

We believe that our employees are our greatest asset. NVIDIA offers extensive professional development opportunities, wellness programs, and a flexible work environment. We support work-life balance to ensure our team members thrive both personally and professionally.

---

## Join Us

### Careers at NVIDIA

NVIDIA is constantly seeking passionate individuals who are innovators at heart. By joining our team, you will not only take part in exciting projects at the forefront of technology, but also gain the opportunity to grow and develop within the company. 

- **Open Positions:** We offer a range of career paths in engineering, research, sales, and marketing. 
- **Inclusive Workplace:** We welcome diverse backgrounds and perspectives, aiming to create a team that mirrors our global customer base.

If you're ready to push the boundaries of what's possible with us, visit our careers page to explore opportunities.

---

## Connect with Us

For more information about our products and corporate initiatives, please visit [NVIDIA's Official Website](https://www.nvidia.com/) or reach out through our social media channels. We look forward to engaging with you!

---

Thank you for considering NVIDIA—where innovation and creativity meet to shape the future.


In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>